## Insights

In [5]:
import pandas as pd
import numpy as np

***Paso 1:*** Cargar la data.

In [72]:
agents = pd.read_csv('agents.csv.gz', compression='gzip', 
                                 header=0, sep=',', quotechar='"')

agents.head(5)

,id,dev_id,read_time,time_stamp,speed,grp_name,localidad,latitude,longitude,devicegroupid
0,1641013210,868033050090275,2022-01-01 00:00:10,2022-01-01 00:00:04,5.700076,S60,PUENTE ARANDA,4.615435,-74.10485,6
1,1641013811,868033050090275,2022-01-01 00:10:11,2022-01-01 00:00:04,5.700076,S60,PUENTE ARANDA,4.615435,-74.10485,6
2,1641014411,868033050090275,2022-01-01 00:20:11,2022-01-01 00:00:04,5.700076,S60,PUENTE ARANDA,4.615435,-74.10485,6
3,1641015012,868033050090275,2022-01-01 00:30:12,2022-01-01 00:00:04,5.700076,S60,PUENTE ARANDA,4.615435,-74.10485,6
4,1641015613,868033050090275,2022-01-01 00:40:13,2022-01-01 00:00:04,5.700076,S60,PUENTE ARANDA,4.615435,-74.10485,6


In [96]:
incidents = pd.read_csv('incidents.csv.gz', compression='gzip', 
                                 header=0, sep=',', quotechar='"')

incidents.head(5)


,id,source,type,state,priority,gravity,class,object,latitude,longitude,location,broker_id,implicated,address,zone_id,orient,incident_time,implicated_type,injured_count,dead_count
0,10667,NUSE,Choque,Cerrado,2.0,Con lesionado,Siniestro,NaN,4.600568,-74.178246,BOSA,NaN,2.0,Calle 59 SUR Carrera 77K BIS A,Zona 2,Oriente - Occidente,2021-12-31 23:40:00,Motocicleta,1.0,0.0
1,10667,NUSE,Choque,Cerrado,2.0,Con lesionado,Siniestro,NaN,4.600568,-74.178246,BOSA,NaN,2.0,Calle 59 SUR Carrera 77K BIS A,Zona 2,Oriente - Occidente,2021-12-31 23:40:00,Motocicleta,1.0,0.0
2,10668,NUSE,Choque,Cerrado,2.0,Con lesionado,Siniestro,NaN,4.619570,-74.139442,KENNEDY,Av Boyacá NS,2.0,Av. Boyacá Calle 15F SUR,Zona 2,Norte - sur,2021-12-31 11:38:00,Automóvil,0.0,0.0
3,10668,NUSE,Choque,Cerrado,2.0,Con lesionado,Siniestro,NaN,4.619570,-74.139442,KENNEDY,Av Boyacá NS,2.0,Av. Boyacá Calle 15F SUR,Zona 2,Norte - sur,2021-12-31 11:38:00,Motocicleta,1.0,0.0
4,10669,NUSE,Choque,Cerrado,2.0,Con lesionado,Siniestro,NaN,4.539066,-74.135452,CIUDAD BOLIVAR,NaN,2.0,Calle 76A SUR Carrera 15,Zona 2,Oriente - Occidente,2022-01-01 00:20:00,Automóvil,0.0,0.0


***Paso 2:*** Limpiar la data

In [47]:
camposNaN = pd.Series(incidents.isna().any())
camposNaN = camposNaN[camposNaN != False].index

print(camposNaN)

#Estrategia de limpieza 1: DropNA / Rechazado, pasa de 114260 registros a 314.
#incidents.dropna(inplace=True)

#Estrategia de limpieza 2: Interpolación / Verificar

Index(['type', 'priority', 'gravity', 'object', 'broker_id', 'implicated',
       'implicated_type', 'injured_count', 'dead_count'],
      dtype='object')


***Paso 3:*** Feature Engineering

In [48]:
incidents["hour"] = pd.to_datetime(incidents["incident_time"]).dt.hour

In [73]:
agents["hour"] = pd.to_datetime(agents["time_stamp"]).dt.hour

***Paso 4:*** Generación de indicadores

In [77]:
#El mayor tipo de accidente
incidents["type"].value_counts()

Choque                       101378
Autolesiones                   3282
Mantenimiento correctivo       2137
Mantenimiento preventivo       1507
Choque contra objeto fijo      1460
Falla de energía                831
Caída de ocupante               533
Volcamiento                     291
Caída de árbol                  265
Reprogramación de equipo        237
Otros                           159
Objetos en vía                  159
Empozamiento o inundación       114
Mancha en vía                   106
Aprisionamiento                  72
Choque con volcamiento           63
Estudiantes                      48
Incendio vehicular               38
Caída poste                      34
Materiales peligrosos            26
Posteria semáforos               23
Conductores                      22
Falla por sieniestro             19
Profesores                        6
Taxistas                          3
Name: type, dtype: int64

In [98]:
#Las localidalides con mayor cantidad de accidentes
incidents["location"].value_counts()

KENNEDY               15500
SUBA                  13225
ENGATIVA              11883
USAQUEN                8880
FONTIBON               8270
PUENTE ARANDA          6816
BOSA                   5777
CHAPINERO              5628
CIUDAD BOLIVAR         5567
BARRIOS UNIDOS         5211
TEUSAQUILLO            5197
RAFAEL URIBE URIBE     4011
LOS MARTIRES           3647
SAN CRISTOBAL          3440
SANTA FE               3115
TUNJUELITO             2869
ANTONIO NARIÑO         2418
USME                   2269
LA CANDELARIA           462
LOCATION MISSING         75
Name: location, dtype: int64

In [100]:
#El mayor tipo de accidente por zona
def mayor_tipo_accidente_por_zona():
    s = incidents[["type","location"]].value_counts()
    df = pd.DataFrame(s)
    df.reset_index(inplace = True)
    return df[df["type"] == "Autolesiones"],"Esto es un insight"
mayor_tipo_accidente_por_zona()

(             type            location    0
 18   Autolesiones             KENNEDY  462
 20   Autolesiones                SUBA  349
 21   Autolesiones            ENGATIVA  301
 22   Autolesiones             USAQUEN  249
 23   Autolesiones       PUENTE ARANDA  223
 25   Autolesiones            FONTIBON  204
 29   Autolesiones         TEUSAQUILLO  184
 30   Autolesiones      CIUDAD BOLIVAR  182
 34   Autolesiones                BOSA  161
 43   Autolesiones           CHAPINERO  132
 44   Autolesiones            SANTA FE  131
 45   Autolesiones  RAFAEL URIBE URIBE  129
 46   Autolesiones      BARRIOS UNIDOS  128
 49   Autolesiones       SAN CRISTOBAL  117
 55   Autolesiones        LOS MARTIRES   99
 66   Autolesiones                USME   79
 72   Autolesiones      ANTONIO NARIÑO   72
 80   Autolesiones          TUNJUELITO   64
 155  Autolesiones       LA CANDELARIA   14
 290  Autolesiones    LOCATION MISSING    2,
 'Esto es un insight')

In [94]:
#El mayor tipo de accidente por hora
incidents[["hour","type"]].value_counts().sort_values().sort_index(kind='mergesort')

hour  type                    
0     Autolesiones                  88
      Caída de ocupante              6
      Caída de árbol                 2
      Choque                      1194
      Choque con volcamiento         2
                                  ... 
23    Materiales peligrosos          1
      Objetos en vía                 1
      Otros                          4
      Reprogramación de equipo       4
      Volcamiento                   11
Length: 458, dtype: int64

In [65]:
#Zonas con mayor accidentalidad por hora
incidents[["location","hour"]].value_counts().sort_values().sort_index(kind='mergesort')

location        hour
ANTONIO NARIÑO  0        20
                1        20
                2        13
                3        18
                4        23
                       ... 
USME            19      156
                20       97
                21       82
                22       74
                23       75
Length: 473, dtype: int64

In [76]:
#Extraer las localidades con mayor presencia de agentes por hora
agents[["localidad","hour"]].value_counts().sort_values().sort_index(kind='mergesort')

localidad       hour
ANTONIO NARIÑO  0        295
                1        272
                2        198
                3        187
                4        285
                        ... 
USME            19      3797
                20      2792
                21      2173
                22      1448
                23       818
Length: 459, dtype: int64